# Calculate and Save Average Ground Truth from Multi Expert Label(Answer)  
- Bring each Dataset and calc. average label   
- Save "subname , trial , actFrames " 

In [1]:
import pandas as pd
import os,re

homeDir = "/DockerProjects/walkCAM/tug/temporalCNN_wLabel"
os.chdir(homeDir);

def get_frame_number(name):
    if not name:
        return 0
    x = [int(s) for s in name[:-4].split('_') if s.isdigit()]
    return x[0]
    
    
lpf_order = 4
cut_off = 0.3
dataset_type = 'stroke' # stroke -3 ,  norm60 -22,  norm20 -1



selected_LPF = "c{0}o{1}".format(cut_off,lpf_order).replace(".","-")
print(" --------- Seleced Type: LPF (4th) Cutoff ",cut_off,"Hz")

rootDir = "/DockerProjects/Dataset/TUG/trainSet/HMMpaper/mixLabel" 
folder1 = 'LSTM2HMM_saveResults_illness_KYH'
folder2 = 'LSTM2HMM_saveResults_illness_KHJ'


path = '/DockerProjects/Dataset/TUG/trainSet/HMMpaper/mixLabel' + '/{folder}/0_sideView/'+dataset_type

print(path , "\t Target_type: ", dataset_type)




 --------- Seleced Type: LPF (4th) Cutoff  0.3 Hz
/DockerProjects/Dataset/TUG/trainSet/HMMpaper/mixLabel/{folder}/0_sideView/stroke 	 Target_type:  stroke


In [2]:
dir_tuples = [tuple for tuple in os.walk(path.format(folder=folder1)) if not tuple[1] and (any('label') in file for file in tuple[2])]

li = []
except_trials = 0
for tuple in dir_tuples:
    seperated_path = tuple[0].split('/')
    
    subject = seperated_path[-2]
    trial = seperated_path[-1]
    csvname = 'label_'+subject+'_'+trial+'.csv'
    try:
        path_kyh = os.path.join(tuple[0], csvname)
        df = pd.read_csv(path_kyh)
        path_khj = path_kyh.replace(folder1, folder2)
        df_from_each_file = (pd.read_csv(f) for f in (path_khj,path_kyh))
        concatenated_df   = pd.concat(df_from_each_file, ignore_index=True).applymap(get_frame_number)
        concatenated_df['subject'] = subject
        concatenated_df['trial'] = trial
        li.append(concatenated_df)
    except:
        except_trials +=1
        print("Except File List (Subject, Trial): ", subject,"_", trial)
        continue

print("# of exception files: ", except_trials)

saveMeanCsv_path = '/DockerProjects/Dataset/TUG/trainSet/HMMpaper/mixLabel/{selected_LPF}'.format(selected_LPF = selected_LPF)
if not os.path.exists( saveMeanCsv_path):
    os.makedirs(saveMeanCsv_path)
pd.concat(li, axis=0, ignore_index=True).groupby(['subject','trial'], as_index=False).mean().round(0).sort_values(by=['subject','trial']).to_csv(
    '/DockerProjects/Dataset/TUG/trainSet/HMMpaper/mixLabel/{selected_LPF}/{selected_LPF}_mean_{dataset_type}.csv'.format(selected_LPF = selected_LPF, dataset_type=dataset_type),index=False)

Except File List (Subject, Trial):  KJH_TUG_tool _ 03
Except File List (Subject, Trial):  KJH_TUG_tool _ 00
Except File List (Subject, Trial):  KJH_TUG_tool _ 01
Except File List (Subject, Trial):  KJH_TUG_tool _ 04
Except File List (Subject, Trial):  KJH_TUG_tool _ 02
Except File List (Subject, Trial):  KYS_TUG_tool _ 01
Except File List (Subject, Trial):  KYS2_TUG _ 03
# of exception files:  7
